In [ ]:
class RuleBasedModel:
    def __init__(self, data):
        self.data = data.copy()
        self.binary_features = [
            'golden_cross', 'short_uptrend', 'price_above_sma20', 'price_above_sma50',
            'rsi_oversold', 'rsi_overbought', 'macd_bullish', 'roc_positive', 'stoch_oversold',
            'bb_oversold', 'bb_overbought', 'high_volatility',
            'volume_spike', 'volume_confirmation', 'volume_divergence'
        ]
        # Default weights: positive = bullish signal, negative = bearish signal
        self.weights = {
            # Trend (bullish)
            'golden_cross': 2.0,
            'short_uptrend': 1.5,
            'price_above_sma20': 1.0,
            'price_above_sma50': 1.0,
            # Momentum
            'rsi_oversold': 1.5,        # Oversold = buy opportunity
            'rsi_overbought': -1.5,     # Overbought = sell signal
            'macd_bullish': 1.5,
            'roc_positive': 1.0,
            'stoch_oversold': 1.0,
            # Volatility/Reversion
            'bb_oversold': 1.5,         # Mean reversion buy
            'bb_overbought': -1.5,      # Mean reversion sell
            'high_volatility': -0.5,    # High vol = reduce exposure
            # Volume
            'volume_spike': 0.5,
            'volume_confirmation': 1.5, # Price up + volume = strong
            'volume_divergence': -1.5   # Price down + volume = weak
        }
        self.results = None
    
    def run_pipeline(self):
        self._calculate_scores()
        self._generate_signals()
        self._evaluate_performance()
    
    def _apply_by_ticker(self, func):
        return self.data.groupby(level='ticker', group_keys=False).apply(func)
    
    def _calculate_scores(self):
        """Calculate weighted score from binary features"""
        self.data['rule_score'] = sum(
            self.data[feat] * self.weights[feat] for feat in self.binary_features
        )
        # Normalize to [-1, 1] range
        max_pos = sum(w for w in self.weights.values() if w > 0)
        max_neg = abs(sum(w for w in self.weights.values() if w < 0))
        self.data['rule_score_norm'] = self.data['rule_score'].apply(
            lambda x: x / max_pos if x > 0 else x / max_neg if x < 0 else 0
        )
    
    def _generate_signals(self):
        """Convert scores to trading signals: 1=long, 0=neutral, -1=short"""
        # Thresholds for signal generation
        long_threshold = 0.2
        short_threshold = -0.2
        
        self.data['signal'] = 0
        self.data.loc[self.data['rule_score_norm'] > long_threshold, 'signal'] = 1
        self.data.loc[self.data['rule_score_norm'] < short_threshold, 'signal'] = -1
        
        # Strategy return: signal * next day's return (signal made today, return tomorrow)
        self.data['strategy_return'] = self.data['signal'] * self.data['target']
    
    def _evaluate_performance(self):
        """Calculate performance metrics per ticker and overall"""
        def calc_metrics(df):
            df = df.dropna(subset=['strategy_return', 'target'])
            if len(df) == 0:
                return pd.Series()
            
            strat_ret = df['strategy_return']
            buy_hold = df['target']
            
            # Annualized metrics
            trading_days = 252
            
            # Cumulative returns
            strat_cum = (1 + strat_ret).prod() - 1
            bh_cum = (1 + buy_hold).prod() - 1
            
            # Sharpe ratio (annualized)
            sharpe = (strat_ret.mean() / strat_ret.std()) * np.sqrt(trading_days) if strat_ret.std() > 0 else 0
            
            # Max drawdown
            cum_returns = (1 + strat_ret).cumprod()
            rolling_max = cum_returns.expanding().max()
            drawdown = (cum_returns - rolling_max) / rolling_max
            max_dd = drawdown.min()
            
            # Win rate
            wins = (strat_ret > 0).sum()
            total_trades = (df['signal'] != 0).sum()
            win_rate = wins / total_trades if total_trades > 0 else 0
            
            return pd.Series({
                'total_return': strat_cum,
                'buy_hold_return': bh_cum,
                'sharpe_ratio': sharpe,
                'max_drawdown': max_dd,
                'win_rate': win_rate,
                'n_trades': total_trades,
                'n_days': len(df)
            })
        
        # Per-ticker results
        self.results = self.data.groupby(level='ticker').apply(calc_metrics)
        
        # Print summary
        print("=" * 60)
        print("RULE-BASED MODEL PERFORMANCE")
        print("=" * 60)
        for ticker in self.results.index:
            r = self.results.loc[ticker]
            print(f"\n{ticker}:")
            print(f"  Strategy Return: {r['total_return']*100:>8.2f}%  |  Buy & Hold: {r['buy_hold_return']*100:>8.2f}%")
            print(f"  Sharpe Ratio:    {r['sharpe_ratio']:>8.2f}   |  Max Drawdown: {r['max_drawdown']*100:>7.2f}%")
            print(f"  Win Rate:        {r['win_rate']*100:>8.1f}%  |  Trades: {int(r['n_trades'])}")
        
        # Overall
        overall = self.data.dropna(subset=['strategy_return'])
        overall_sharpe = (overall['strategy_return'].mean() / overall['strategy_return'].std()) * np.sqrt(252)
        print(f"\n{'='*60}")
        print(f"OVERALL SHARPE: {overall_sharpe:.3f}")

# Run the rule-based model
rb_model = RuleBasedModel(features_data)
rb_model.run_pipeline()